In [2]:
import sys
sys.path.append('../')

### Part - 1 Basics

In [ ]:
from trading_env import DataSource, TradeBroker, TradingEnv
import trading_env

In [ ]:
d = DataSource("../../Data/DAT_MT_EURUSD_M1_2021.csv", end_index=30)
tb = TradeBroker(d, initial_balance=10, price_history_length=5)
env = TradingEnv(tb)

In [ ]:
env.reset()

In [ ]:
for i in range(40):
    obs, reward, done, info = env.step(2)
    if done is True: 
        break

In [ ]:
import random

In [ ]:
d = DataSource("../../Data/DAT_MT_EURUSD_M1_2021.csv",)
tb = TradeBroker(d, initial_balance=1000, price_history_length=10)
env = TradingEnv(tb)
env.reset()
trading_env.logger.setLevel("CRITICAL")
from tqdm import tqdm
total_reward = 0
for i in tqdm(range(20000)):
    obs, reward, done, info = env.step(random.choice([0,1,2]))
    total_reward += reward
    if done is True: 
        break
    

In [ ]:
info

In [ ]:
total_reward

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(tb.balance_history)

### Part - 2: Replay Buffer


In [3]:
from main import test_buffer_working

In [4]:
csv_file = "../../Data/DAT_MT_EURUSD_M1_2021.csv"
buffer = test_buffer_working(csv_file=csv_file, start_index=0, end_index=2000)

2024-03-09 03:27:03,405 - trading_env - DEBUG - Buying 1 share at 1.22322
2024-03-09 03:27:03,407 - trading_env - DEBUG - Selling 1 share at 1.22315
2024-03-09 03:27:03,410 - trading_env - DEBUG - No shares to sell
2024-03-09 03:27:03,413 - trading_env - DEBUG - Buying 1 share at 1.22341
2024-03-09 03:27:03,417 - trading_env - DEBUG - Selling 1 share at 1.2237
2024-03-09 03:27:03,421 - trading_env - DEBUG - Buying 1 share at 1.2236
2024-03-09 03:27:03,425 - trading_env - DEBUG - Selling 1 share at 1.22354
2024-03-09 03:27:03,428 - trading_env - DEBUG - Holding at 1.22339
2024-03-09 03:27:03,431 - trading_env - DEBUG - No shares to sell
2024-03-09 03:27:03,434 - trading_env - DEBUG - Holding at 1.22333
2024-03-09 03:27:03,438 - trading_env - DEBUG - No shares to sell
2024-03-09 03:27:03,441 - trading_env - DEBUG - Holding at 1.22333
2024-03-09 03:27:03,444 - trading_env - DEBUG - Buying 1 share at 1.22366
2024-03-09 03:27:03,447 - trading_env - DEBUG - Buying 1 share at 1.22392
2024-03-

In [12]:
buffer.memory[-1]

(array([[0.99850902, 0.99853359, 0.99829602, 0.99833698, 0.24576667],
        [0.99834517, 0.99872201, 0.99823867, 0.99845167, 0.24576667],
        [0.99845986, 0.99859094, 0.99830421, 0.99839432, 0.24576667],
        [0.99839432, 0.9987384 , 0.99834517, 0.99873021, 0.32768889],
        [0.9987384 , 0.99901693, 0.99865648, 0.99886947, 0.40961112],
        [0.99886947, 0.99911524, 0.99885309, 0.99900055, 0.40961112],
        [0.99899236, 0.99903332, 0.99885309, 0.99898416, 0.49153334],
        [0.99899236, 0.9992627 , 0.99895959, 0.99909886, 0.49153334],
        [0.99912343, 0.99953304, 0.99911524, 0.99938558, 0.57345556],
        [0.9993692 , 0.99999181, 0.99934462, 0.99972146, 0.57345556],
        [0.99972966, 0.99988531, 0.9994757 , 0.999787  , 0.65537778],
        [0.99977881, 0.99983616, 0.9994757 , 0.99955762, 0.57345556],
        [0.99955762, 1.        , 0.99948389, 0.99995085, 0.57345556],
        [0.99994265, 0.99994265, 0.99952485, 0.99953304, 0.49153334],
        [0.99951666,

In [58]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Set the seed for the random number generator
np.random.seed(0)
torch.manual_seed(0)

class QNetworkCNN(nn.Module):
	
	def __init__(self, input_shape, n_actions):
		super(QNetworkCNN, self).__init__()
		self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=32, kernel_size=3, padding=1)
		self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
		self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
		self.fc1 = nn.Linear(64*input_shape[1]*input_shape[2], 512)
		self.fc2 = nn.Linear(512, n_actions)
	
	def forward(self, x):
		x = F.relu(self.conv1(x))
		print(f"Conv1: {x.shape}")
		x = F.relu(self.conv2(x))
		print(f"Conv2: {x.shape}")
		x = F.relu(self.conv3(x))
		print(f"Conv3: {x.shape}")
		x = x.view(x.size(0), -1)
		print(f"View: {x.shape}")
		x = F.relu(self.fc1(x))
		print(f"FC1: {x.shape}")
		return self.fc2(x)


In [69]:
cnn = QNetworkCNN((1,24,5), 3)

In [70]:
inp = buffer[0][0]
print(inp.shape)

(24, 5)


In [71]:
inp = torch.from_numpy(inp).unsqueeze(0).float()

In [72]:
inp.shape

torch.Size([1, 24, 5])

In [73]:
cnn.forward(inp)

Conv1: torch.Size([32, 24, 5])
Conv2: torch.Size([64, 24, 5])
Conv3: torch.Size([64, 24, 5])
View: torch.Size([64, 120])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x120 and 7680x512)

In [57]:
25*5

125